In [1]:
#from google.colab import drive
import numpy as np
import os
from os import listdir
from os.path import isfile, join, splitext
from PIL import Image, ImageOps
import numpy as np
import random
import math
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.regularizers import l1
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input
from matplotlib import pyplot
import h5py

In [ ]:
#If you didn't run the previous code that do the preprocessing -> run this part of code please
#DATASET USED FOR THE MODEL FOR TEMPO ESTIMATION
!gdown --id 1nzep5kiiLNjjoFgoMa-gvLSbIQIr18yP
!gdown --id 1GemCMmnmkiubcf77Q_uD4xuM90k7rGkx

path=['Image_spec_key.zip','Annotations_key.zip']
path_images=["Image_spec_key/GS","Image_spec_key/GTZAN"]
path_ann=["Annotations_key/GS","Annotations_key/GTZAN"]
from zipfile import ZipFile
for p in path:
    with ZipFile(p, 'r') as zipp:
        # printing all the contents of the zip file
        #zip.printdir()
        print('Extracting all the files now...')
        zipp.extractall()
        print('Done!')
flag=0

In [4]:
#If you have done the preprocessing using the previous notebook, please run this part of code
path_images=["Dataset/Key/GS/Image_spec","Dataset/Key/GTZAN/Image_spec"]
path_ann=["Dataset/Key/GS/Ann","Dataset/Key/GTZAN/Ann"]
flag=1

In [6]:
def split_train_test_validation(path_label,path_img):
    f_l=[]
    f_i=[]


    for i,l in zip(path_images,path_ann):
        f_l.extend([os.path.join(root, name) for root, dirs, files in os.walk(l) for name in files if name.endswith((".txt"))])
        f_i.extend([os.path.join(root, name) for root, dirs, files in os.walk(i) for name in files if name.endswith((".h5"))]) 

    if flag==0:

        f_l=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_l]
        f_i=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_i]
    if flag==1:
        f_l=[f.split('/')[2]+"/"+(f.split('/')[-1]).split('\\')[1] for f in f_l]
        f_i=[f.split('/')[2]+"/"+(f.split('/')[-1]).split('\\')[1] for f in f_i]
    f_l= np.asarray(f_l)
    f_i= np.asarray(f_i)
    print(f_l)
    print(f_i)
    print("Sorting both images and annotations...")
    for n in range(f_l.shape[0]):
        while True:
            
            s=splitext(f_l[n])[0]
            #print(s)
            if "lerch" in s:
                s=s[:len(s)-6]

            res=np.where(f_i==f"{str(s)}.h5")
            if(n==res[0]):
                break
            f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
    c=list(zip(f_l,f_i))
    random.shuffle(c)
    f_l,f_i=zip(*c)
    if flag==1:
        f_l=["Dataset/Key/"+f.split('/')[0]+'/Ann/'+f.split('/')[1] for f in f_l]
        f_i=["Dataset/Key/"+f.split('/')[0]+'/Image_spec/'+f.split('/')[1] for f in f_i]
    images=[]
    ann=[]
    print("Processing the images...")
    for f in f_i:
        h5f = h5py.File(f"{f}",'r')
        b = h5f['d'][:]
        images.append(np.asarray(b))
        print(b.shape)
    print("Processing the annotations")
    
    for an in f_l:
        
        with open(an) as f:
            ann.append(int(f.readlines()[0]))
    test_x=np.asarray(images[0:math.ceil(len(f_i)*0.2)])
    test_file=f_i[:math.ceil(len(f_i)*0.2)]
    test_y=np.array(ann[0:math.ceil(len(f_i)*0.2)])
    train_x=np.asarray(images[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
    train_y=np.array(ann[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
    val_x=np.asarray(images[len(test_x)+len(train_x):])
    val_y=np.array(ann[len(test_x)+len(train_x):])
    train_y = to_categorical(train_y, 24)
    test_y = to_categorical(test_y, 24) 
    val_y= to_categorical(val_y, 24)
    print(f"Split a total of {len(f_i)} samples into three sets with {len(train_x)} training, {len(val_x)} validation and {len(test_x)} test samples.")
    return test_x,test_y,train_x,train_y,val_x,val_y,test_file


x_test,y_test,x_train,y_train,x_val,y_val,test_file=split_train_test_validation(path_ann,path_images)


['GS/1004923.LOFI_1.txt' 'GS/1004923.LOFI_2.txt' 'GS/1004923.LOFI_3.txt'
 ... 'GTZAN/rock.00098_2.txt' 'GTZAN/rock.00099_1.txt'
 'GTZAN/rock.00099_2.txt']
[]
Sorting both images and annotations...
GS/1004923.LOFI_1


<ipython-input-6-e8057eff73bb>:30: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res=np.where(f_i==f"{str(s)}.h5")
<ipython-input-6-e8057eff73bb>:31: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if(n==res[0]):


IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input


model = tf.keras.Sequential([
    layers.Conv2D(16, (5, 5), activation='relu', input_shape=(256, 40,1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)), 
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.Dense(24, activation='softmax')
])
model.summary()
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.055),loss='categorical_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train, epochs=75, batch_size=132,validation_data=(x_val, y_val),callbacks=[es,mc])


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_3. Consider increasing the input size. Received input shape [None, 23, 1, 16] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')

In [ ]:
#the accuracy is defined as the readme explain:
#Relative fifth: error of 50% (Ex. Label: C major , Prediction: G major)
#Relative minor/major: error of 75% (Ex. Label: C major, Prediction: A minor)
#Parallel minor/major: error of 90% (Ex. Label: C major, Prediciotn: C minor) 
def get_accuracy(model, true_x, true_y): 
    res = model.predict(true_x)
    res = np.argmax(res, axis=-1)
    acc = 0
    for i in range(len(true_y[:, 0])):
        r=res[i]
        t=np.where(true_y[i,:]==1)[0]
        if r==t:
          
          acc+=1
        elif r==(t+7)% (24//2):
          acc+=0.5
        elif r==(t-3) or r==(t+9)%24 or r==(t+3)%(24//2):
          acc+=0.25
        elif (r==(t+12)%24):
          acc+=0.1
        else:
            pass
    tot = len(true_y[:,0])
    print('True - total', acc, tot)
    print('acc: {}'.format((acc/tot)))

    
#This function is implemented in order to have the training dataset distinguished for the original dataset, indeed
#the output are the ExtendedBallroom and Giantsteps dataset
def get_dataset(label,img,test_file):
  f_l=[f for f in listdir("Annotations") if isfile(join("Annotations", f))]
  f_i=[f for f in listdir("Image_spec2") if isfile(join("Image_spec2", f))]
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  for n in range(f_l.shape[0]):
    print(n)
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.where(f_i==f"{str(s)}.jpeg")
      if(n==res[0]):
        break
      f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
  c=list(zip(f_l,f_i))
  giant_i=[]
  gtzan_i=[]
  gtzan_l=[]
  giant_l=[]
  lofi="LOFI"
  words=['blues','classical','country','disco','hiphop','jazz','metal','rock','reggae','pop']
  for f in f_i:
    if f in test_file: 
      if lofi in f:
        idx=np.where(f_i == f)
        giant_i.append(f)
        giant_l.append(f_l[idx])
      for w in words:
        if w in f:
          idx=np.where(f_i == f)
          gtzan_i.append(f)
          gtzan_l.append(f_l[idx])
  gt_i=[]
  gt_l=[]
  gi_i=[]
  gi_l=[]
  for g1 in gtzan_i:
    gt_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g1}").resize((100,100)))))
  for g2 in giant_i:  
    gi_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g2}").resize((100,100)))))
  for g1 in gtzan_l:
    with open(f"Annotations/{g1[0]}") as f:
      gt_l.append(int(f.readlines()[0]))
  for g2 in giant_l:
    with open(f"Annotations/{g2[0]}") as f:
      gi_l.append(int(f.readlines()[0]))
  gt_i=np.asarray(gt_i)
  gt_l=np.asarray(gt_l)
  gi_i=np.asarray(gi_i)
  gi_l=np.asarray(gi_l)
  return gt_i,gt_l,gi_i,gi_l
gtzan_i,gtzan_l,giant_i,giant_l=get_dataset(path_ann,path_images,test_file)


In [ ]:
get_accuracy(model,giant_i,giant_l)
get_accuracy(model,gtzan_i,gtzan_l)